In [34]:
import os
import tiktoken
import openai
import pinecone

In [20]:
from langchain.document_loaders import UnstructuredEPubLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [36]:
openai_api_key = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [21]:
loader = UnstructuredEPubLoader("./Epub/arthur-conan-doyle_the-casebook-of-sherlock-holmes.epub")

In [23]:
tokenizer = tiktoken.get_encoding('p50k_base')

In [24]:
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)

In [25]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,
    chunk_overlap=20,
    length_function=tiktoken_len,
    separators=["\n\n", "\n", " ", ""]
)

In [26]:
data = loader.load()
docs = text_splitter.split_documents(data)

In [42]:
from uuid import uuid4
from tqdm.auto import tqdm

chunks = []

for idx, record in enumerate(tqdm(docs)):
    #texts = text_splitter.split_text(record.page_content)
    uuid_value = uuid4()
    chunks.extend([{
        
        'id': str(uuid_value),
        'text': record.page_content,
        'chunk': idx,
        #'source':record.metadata['source']
        
    } ])

100%|██████████| 386/386 [00:00<00:00, 128004.53it/s]


In [40]:

index_name = 'gpt-4-langchain-docs'

# initialize connection to pinecone
pinecone.init(
    api_key=pinecone_api_key,  # app.pinecone.io (console)
    environment="us-central1-gcp"  # next to API key in console
)
index = pinecone.Index(index_name)

In [45]:
from tqdm.auto import tqdm
import datetime
from time import sleep

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(chunks), batch_size)):
    # find end of batch
    i_end = min(len(chunks), i+batch_size)
    meta_batch = chunks[i:i_end]
    # get ids
    ids_batch = [x['id'] for x in meta_batch]
    # get texts to encode
    texts = [x['text'] for x in meta_batch]
    # create embeddings (try-except added to avoid RateLimitError)
    try:
        res = openai.Embedding.create(input=texts, engine=embed_model)
    except:
        done = False
        while not done:
            sleep(5)
            try:
                res = openai.Embedding.create(input=texts, engine=embed_model)
                done = True
            except:
                pass
    embeds = [record['embedding'] for record in res['data']]
    # cleanup metadata
    meta_batch = [{
        'text': x['text'],
        'chunk': x['chunk'],
        
        
    } for x in meta_batch]
    to_upsert = list(zip(ids_batch, embeds, meta_batch))
    # upsert to Pinecone
    index.upsert(vectors=to_upsert)


100%|██████████| 4/4 [00:57<00:00, 14.37s/it]


In [55]:
word_f =  data[0].page_content

In [56]:
word_f

'Imprint\n\nThis ebook is the product of many hours of hard work by volunteers\r\nfor Standard Ebooks, and builds\r\non the hard work of other literature lovers made possible by the public\r\ndomain.\n\nThis particular ebook is based on a transcription from Project Gutenberg\r\nAustralia and on digital scans from the Internet\r\nArchive.\n\nThe source text and artwork in this ebook are believed to be in the\r\nUnited States public domain; that is, they are believed to be free of\r\ncopyright restrictions in the United States. They may still be\r\ncopyrighted in other countries, so users located outside of the United\r\nStates must check their local laws before using this ebook. The creators\r\nof, and contributors to, this ebook dedicate their contributions to the\r\nworldwide public domain via the terms in the CC0 1.0\r\nUniversal Public Domain Dedication. For full license information,\r\nsee the Uncopyright at the end of this\r\nebook.\n\nStandard Ebooks is a volunteer-driven project

In [59]:
word_f = word_f.replace('\n',' ' )
word_f = word_f.replace('\r',' ')

In [62]:
word_f
len(word_f)

454862

In [64]:
path = './txt/f/arthur-conan-doyle_the-casebook-of-sherlock-holmes.txt'
f = open(path, 'w')
print(word_f, file=f)

In [65]:
from langchain.text_splitter import CharacterTextSplitter

IndexError: list index out of range